In [13]:
import numpy as np
from qutip import basis, sigmax, Qobj, expect

def binary_entropy(x):
    if x == 0 or x == 1:
        return 0.0
    return -x*np.log2(x) - (1-x)*np.log2(1-x)

def simulate_bb84(num_bits=10000, bit_flip_prob=0.02):

    bits        = np.random.randint(0, 2, size=num_bits)
    alice_bases = np.random.randint(0, 2, size=num_bits)
    H = Qobj([[1,1],[1,-1]])/np.sqrt(2)


    states = []
    for b, base in zip(bits, alice_bases):
        psi = basis(2, b)
        if base == 1:
            psi = H * psi
        states.append(psi)

    flips = np.random.rand(num_bits) < bit_flip_prob
    noisy = [sigmax()*psi if f else psi for psi, f in zip(states, flips)]


    bob_bases = np.random.randint(0, 2, size=num_bits)
    P0 = basis(2,0)*basis(2,0).dag()
    P1 = basis(2,1)*basis(2,1).dag()
    plus  = H*basis(2,0)
    minus = H*basis(2,1)
    Pplus  = plus*plus.dag()
    Pminus = minus*minus.dag()

    received = []
    for psi, base in zip(noisy, bob_bases):
        if base == 0:
            probs = [expect(P0, psi), expect(P1, psi)]
        else:
            probs = [expect(Pplus, psi), expect(Pminus, psi)]
        received.append(np.random.choice([0,1], p=probs))
    received = np.array(received)
    mask      = (alice_bases == bob_bases)
    sift_sent = bits[mask]
    sift_recv = received[mask]


    qber = np.mean(sift_sent != sift_recv)
    rate = max(0.0, 1 - 2*binary_entropy(qber))

    return qber, rate

if __name__ == "__main__":
    qber, key_rate = simulate_bb84(num_bits=20000, bit_flip_prob=0.03)
    print(f"Simulated QBER: {qber*100:.2f}%")
    print(f"Asymptotic key rate r = 1 - 2*h(QBER): {key_rate:.4f} bits/sifted bit")




Simulated QBER: 1.52%
Asymptotic key rate r = 1 - 2*h(QBER): 0.7725 bits/sifted bit


In [25]:
H = Qobj([[1,1],[1,-1]])/np.sqrt(2)

states = []
psi = basis(2, 1)
psi = H * psi
# if base == 1:
print(psi)
# for b, base in zip(bits, alice_bases):
#         psi = basis(2, b)
#         if base == 1:
#             psi = H * psi
#         states.append(psi)

Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[ 0.70710678]
 [-0.70710678]]


In [12]:
import numpy as np
from qutip import basis, Qobj, expect, sigmax, sigmay, sigmaz, qeye

def binary_entropy(x):
    """Binary entropy h(x) = -x log2(x) - (1−x) log2(1−x)."""
    if x == 0 or x == 1:
        return 0.0
    return -x * np.log2(x) - (1 - x) * np.log2(1 - x)

def depolarizing_channel_manual(rho, p):

    K0 = np.sqrt(1 - 3*p/4) * qeye(2)
    K1 = np.sqrt(p/4) * sigmax()
    K2 = np.sqrt(p/4) * sigmay()
    K3 = np.sqrt(p/4) * sigmaz()

    return K0 * rho * K0.dag() + K1 * rho * K1.dag() + K2 * rho * K2.dag() + K3 * rho * K3.dag()

def simulate_bb84_with_depol_manual(num_bits=10000, p_dep=0.02):
    bits = np.random.randint(0, 2, size=num_bits)
    alice_bases = np.random.randint(0, 2, size=num_bits)
    H = Qobj([[1, 1], [1, -1]]) / np.sqrt(2)

    states = []
    for bit, base in zip(bits, alice_bases):
        psi = basis(2, bit)
        if base == 1:
            psi = H * psi
        states.append(psi)


    noisy_states = [depolarizing_channel_manual(psi.proj(), p_dep) for psi in states]

    bob_bases = np.random.randint(0, 2, size=num_bits)
    P0 = basis(2, 0)*basis(2, 0).dag()
    P1 = basis(2, 1)*basis(2, 1).dag()
    plus = H * basis(2, 0)
    minus = H * basis(2, 1)
    Pplus = plus * plus.dag()
    Pminus = minus * minus.dag()

    received = []
    for rho, base in zip(noisy_states, bob_bases):
        if base == 0:
            probs = [expect(P0, rho), expect(P1, rho)]
        else:
            probs = [expect(Pplus, rho), expect(Pminus, rho)]
        received.append(np.random.choice([0, 1], p=probs))
    received = np.array(received)

    mask = (alice_bases == bob_bases)
    sift_sent = bits[mask]
    sift_recv = received[mask]

    qber = np.mean(sift_sent != sift_recv)
    key_rate = max(0.0, 1 - 2 * binary_entropy(qber))
    return qber, key_rate


qber, rate = simulate_bb84_with_depol_manual(num_bits=20000, p_dep=0.03)
qber, rate


(np.float64(0.014094362255097961), np.float64(0.7862951345593565))

In [21]:
# !pip install qutip
import numpy as np
from qutip import basis, Qobj, expect, sigmax, qeye

def binary_entropy(x):

    if x == 0 or x == 1:
        return 0.0
    return -x * np.log2(x) - (1 - x) * np.log2(1 - x)

def depolarizing_channel(rho, p):

    return (1 - p) * rho + p * qeye(2) / 2

def bit_flip_channel(rho, p):

    K0 = np.sqrt(1 - p) * qeye(2)
    K1 = np.sqrt(p) * sigmax()
    return K0 * rho * K0.dag() + K1 * rho * K1.dag()

def simulate_bb84_mixed_noise(num_bits=10000, p_bit=0.01, p_dep=0.02):

    bits        = np.random.randint(0, 2, size=num_bits)
    alice_bases = np.random.randint(0, 2, size=num_bits)


    H = Qobj([[1,  1],
              [1, -1]]) / np.sqrt(2)


    states = []
    for b, base in zip(bits, alice_bases):
        psi = basis(2, b)
        if base == 1:
            psi = H * psi
        states.append(psi)


    noisy = []
    for psi in states:
        rho = psi.proj()
        rho_dep = depolarizing_channel(rho, p_dep)
        rho_mix = bit_flip_channel(rho_dep, p_bit)
        noisy.append(rho_mix)


    bob_bases = np.random.randint(0, 2, size=num_bits)
    P0 = basis(2,0)*basis(2,0).dag()
    P1 = basis(2,1)*basis(2,1).dag()
    plus  = H * basis(2,0)
    minus = H * basis(2,1)
    Pplus  = plus * plus.dag()
    Pminus = minus * minus.dag()


    received = []
    for rho, base in zip(noisy, bob_bases):
        if base == 0:
            probs = [expect(P0, rho), expect(P1, rho)]
        else:
            probs = [expect(Pplus, rho), expect(Pminus, rho)]
        received.append(np.random.choice([0,1], p=probs))
    received = np.array(received)

    mask      = (alice_bases == bob_bases)
    sift_sent = bits[mask]
    sift_recv = received[mask]


    qber   = np.mean(sift_sent != sift_recv)
    rate   = max(0.0, 1 - 2 * binary_entropy(qber))
    return qber, rate


if __name__ == "__main__":
    qber, rate = simulate_bb84_mixed_noise(num_bits=30000, p_bit=0.03, p_dep=0)
    print(f"QBER with mixed noise: {qber*100:.2f}%")
    print(f"Key rate: {rate:.4f} bits/sifted bit")


QBER with mixed noise: 1.40%
Key rate: 0.7869 bits/sifted bit


In [23]:
import numpy as np
from qutip import basis, Qobj, expect, sigmax, sigmay, sigmaz, qeye

def binary_entropy(x):

    if x == 0 or x == 1:
        return 0.0
    return -x*np.log2(x) - (1-x)*np.log2(1-x)

def depolarizing_channel(rho, p):

    return (1 - p) * rho + p * qeye(2) / 2

def bit_flip_channel(rho, p):

    K0 = np.sqrt(1-p) * qeye(2)
    K1 = np.sqrt(p)   * sigmax()
    return K0*rho*K0.dag() + K1*rho*K1.dag()

def simulate_b92(num_bits=10000, p_dep=0.02, p_bit=0.01):

    H = Qobj([[1, 1], [1, -1]])/np.sqrt(2)
    states = [basis(2, 0), H*basis(2, 0)]

    bits = np.random.randint(0, 2, size=num_bits)

    rhos = []
    for b in bits:
        psi = states[b]
        rho = psi.proj()
        rho = depolarizing_channel(rho, p_dep)
        rho = bit_flip_channel(rho, p_bit)
        rhos.append(rho)


    plus  = H*basis(2, 0)
    minus = H*basis(2, 1)
    P1   = basis(2,1)*basis(2,1).dag()
    Pm   = minus*minus.dag()

    conclusive, inferred = [], []
    for rho, b in zip(rhos, bits):

        if np.random.rand() < 0.5:
            p_click = expect(P1, rho)
            if np.random.rand() < p_click:

                conclusive.append(True)
                inferred.append(1)
            else:
                conclusive.append(False)
                inferred.append(None)
        else:
            p_click = expect(Pm, rho)
            if np.random.rand() < p_click:

                conclusive.append(True)
                inferred.append(0)
            else:
                conclusive.append(False)
                inferred.append(None)


    mask = np.array(conclusive)
    sent_bits = bits[mask]
    rec_bits  = np.array([b for b in inferred if b is not None])


    qber      = np.mean(sent_bits != rec_bits)
    sift_rate = np.sum(mask)/num_bits
    key_rate  = 1 * max(0.0, 1 - 2*binary_entropy(qber))

    return qber, sift_rate, key_rate

if __name__ == "__main__":
    qber, sift_rate, key_rate = simulate_b92(
        num_bits=30000,
        p_dep=0.03,
        p_bit=0
    )
    print(f"Simulated B92 (mixed noise) → QBER: {qber*100:.2f}%")
    print(f"                Sift rate: {sift_rate*100:.2f}%")
    print(f"                Key rate:  {key_rate:.4f} bits/signal")

Simulated B92 (mixed noise) → QBER: 2.87%
                Sift rate: 25.42%
                Key rate:  0.6241 bits/signal


B92 with bit-flip noise → QBER: 50.54%
Sift rate: 49.99%
Asymptotic key rate: 0.0000 bits/signal
